# Vault Setup in Kubernetes (EKS)

This demo is to show how a Vault cluster can be configured on Kubernetes with a sidecar (busybox)

## Summary of solution

This setup is tested on MacOS and is meant to simulate a distributed setup.  In this demo, we will be going through the following steps:
- Setup a kind K8s cluster (https://kind.sigs.k8s.io/)
- Install and Configure a 3 node Vault cluster with busybox sidecars using the Vault Helm Chart
- Expose the Vault nodes using a NodePort

## Requirements to Run This Demo
You will need Visual Studio Code to be installed with the Jupyter plugin.  To run this notebook in VS Code, chose the Jupyter kernel and then Bash.
- To run the current cell, use Ctrl + Enter.
- To run the current cell and advance to the next, use Shift+Enter.

# Setup Pre-requisites (One-time)

Assumes you have docker installed and brew installed

- https://docs.docker.com/desktop/install/mac-install/
- https://brew.sh/

In [ ]:
# Install kind
brew install kind

In [ ]:
# Install Kubectl CLI
brew install kubernetes-cli

In [ ]:
# Install Helm CLI.  This is used to install the VSO helm chart.
brew install helm

In [ ]:
# Install K9s.  This is a nice console GUI for K8s.  https://k9scli.io/
brew install K9s

# Setup K8s cluster

In [ ]:
# Start a kind cluster 3 nodes for the Vault cluster and 1 node for the Transit Auto-Unseal
# We will be setting up 6 worker nodes 
# Note that the Vault helm chart default affinity settings spreads a Vault setup across different host nodes
# We will be doing a NodePort on port 30000 so kind needs to configure the extraPortMappings to expose port 30000 to the host
kind create cluster --name vault --image kindest/node:v1.28.0 --config - <<EOF
kind: Cluster
apiVersion: kind.x-k8s.io/v1alpha4
nodes:
- role: control-plane
  extraPortMappings:
  - containerPort: 30000
    hostPort: 30000
    listenAddress: "0.0.0.0" # Optional, defaults to "0.0.0.0"
    protocol: tcp # Optional, defaults to tcp
- role: worker
- role: worker
- role: worker
- role: worker
- role: worker
- role: worker
EOF

In [ ]:
# Verify kind containers are running
docker ps
echo
# Show that we have 6 nodes in our K8s cluster
kubectl get nodes

In [ ]:
# Create a new K8s namespace for this demo
## Specify the K8s namespace for the Vault setup
export KUBENAMESPACE=vault-ns

## Delete namespace if it exists
#kubectl delete ns $KUBENAMESPACE

echo "Creating K8s namespace: $KUBENAMESPACE"
kubectl create ns $KUBENAMESPACE

In [ ]:
# Setup Vault Enterprise License in a K8s secret.  Update the path to your license file.
export VAULT_LICENCE=$(cat ../vault.hclic)
#kubectl delete secret vault-ent-license -n $KUBENAMESPACE
kubectl create secret generic vault-ent-license --from-literal="license=${VAULT_LICENCE}" -n $KUBENAMESPACE

In [ ]:
# We will be exposing the vault nodes using a NodePort on port 30000
# vault-active: "true" is commented out.  If included, it will only route to the leader node
kubectl apply -n $KUBENAMESPACE -f - <<EOF
kind: Service
apiVersion: v1
metadata:
  name: port-vault-svc
spec:
  type: NodePort 
  selector:
    app.kubernetes.io/name: "vault"
    app.kubernetes.io/instance: "vault"
    component: server
    #vault-active: "true"
  ports:
    - nodePort: 30000
      port: 8200
      targetPort: 8200
EOF

In [ ]:
# Configure my host to connect to the NodePort for Vault
export VAULT_ADDR=http://localhost:30000

In [ ]:
# Add the HashiCorp repo (Only required for the first time)
helm repo add hashicorp https://helm.releases.hashicorp.com

In [ ]:
# Optional.  Update the repo (Only required when new versions are released)
helm repo update

In [ ]:
# Optional.  This allows you to view the helm charts for vault
helm search repo hashicorp/vault -l

# Setting up a new 3 node Vault Cluster

In [ ]:
# Install a 3 node Vault cluster using the Vault helm chart.  
# This will configure the raft database on PersistentVolumes and also configure raft auto join between the 3 Vault pods.
# For demo purposes, we will be using HTTP.
# See https://developer.hashicorp.com/vault/docs/platform/k8s/helm/configuration for options
helm install vault hashicorp/vault --version 0.28.1 -n $KUBENAMESPACE -f - <<EOF
injector:
  enabled: false
server:
  image:
    repository: hashicorp/vault-enterprise
    tag: latest
  enterpriseLicense:
    secretName: vault-ent-license
  logLevel: trace
  ha:
    enabled: true
    replicas: 3
    raft:
      enabled: true
      setNodeId: true
      config: |
        disable_mlock = true
        ui = true
        listener "tcp" {
          tls_disable = 1
          address = "[::]:8200"
          cluster_address = "[::]:8201"
        }
        storage "raft" {
          # PVC Volume to keep Vault data
          path = "/vault/data"
          # For auto-join to the raft cluster
          retry_join {
            leader_api_addr = "http://vault-0.vault-internal:8200"
          }
          retry_join {
            leader_api_addr = "http://vault-1.vault-internal:8200"
          }
          retry_join {
            leader_api_addr = "http://vault-2.vault-internal:8200"
          } 
        }
  auditStorage:     # Enable audit storage
    enabled: true
  extraContainers:  # Deploy sidecar container
    - name: debug-container
      image: busybox
      command: ["sh", "-c", "while true; do sleep 3600; done"]
      volumeMounts:
        - name: audit
          mountPath: /vault/audit
EOF



In [ ]:
# View installed charts
helm list -A


In [ ]:
# View Vault pods in Vault namespace
#kubectl get pods -n $KUBENAMESPACE -o wide

# Show resources in Vault namespace
kubectl -n $KUBENAMESPACE get all

# Make sure all Vault pods are in Running status before continuing

# Note:
# The containers should start within less than a minute.  If the containers get stuck in ContainerCreating for very long without any errors.
# There could be throttling issues on the DockerHub side.  You might want to kill and restart the kind cluster and try again.

In [ ]:
# Check that all the pods have 2 containers running (vault and debug-container)
kubectl get pods vault-0 -n $KUBENAMESPACE -o jsonpath='{.spec.containers[*].name}'
kubectl get pods vault-1 -n $KUBENAMESPACE -o jsonpath='{.spec.containers[*].name}'
kubectl get pods vault-2 -n $KUBENAMESPACE -o jsonpath='{.spec.containers[*].name}'

In [ ]:
# On first time setup, verify that all Vault nodes are sealed and not initialized.  Initialized = false & Sealed = true
kubectl exec -ti vault-0 -n $KUBENAMESPACE -- vault status
echo
kubectl exec -ti vault-1 -n $KUBENAMESPACE -- vault status
echo
kubectl exec -ti vault-2 -n $KUBENAMESPACE -- vault status

In [ ]:
# Initialize vault-0 pod.  For demo purposes, we will just be generating 1 unseal key.
kubectl exec -ti vault-0 -n $KUBENAMESPACE -- vault operator init -format=json -key-shares=1 -key-threshold=1 > init.json

In [ ]:
# Show the init.json
cat init.json | jq

# Store the Unseal Key and Root Token for use later
export UNSEAL_KEY=$(jq -r '.unseal_keys_b64[]' init.json)
export VAULT_TOKEN=$(jq -r '.root_token' init.json)
echo
echo "Vault Unseal Key: $UNSEAL_KEY"
echo "Vault Root Token: $VAULT_TOKEN"

In [ ]:
# Unseal vault-0 pod.  You should see Sealed = false.  Re-run the command if Sealed is true.
echo "Vault Unseal Key: $UNSEAL_KEY"
kubectl exec -ti vault-0 -n $KUBENAMESPACE -- vault operator unseal $UNSEAL_KEY

In [ ]:
# Unseal vault-1 pod.  You should see Sealed = false.  Re-run the command if Sealed is true.
echo "Vault Unseal Key: $UNSEAL_KEY"
kubectl exec -ti vault-1 -n $KUBENAMESPACE -- vault operator unseal $UNSEAL_KEY

In [ ]:
# Unseal vault-2 pod.  You should see Sealed = false.  Re-run the command if Sealed is true.
echo "Vault Unseal Key: $UNSEAL_KEY"
kubectl exec -ti vault-2 -n $KUBENAMESPACE -- vault operator unseal $UNSEAL_KEY

In [ ]:
# Enable audit on log to write to pod file
vault audit enable file file_path=/vault/audit/vault.log

In [ ]:
# Verify that I can access the vault cluster from the node port
vault secrets list
echo
# Test logging in as root on vault-0 and verify that you can also access vault from the pod
kubectl exec -ti vault-0 -n $KUBENAMESPACE -- vault login $VAULT_TOKEN
echo
kubectl exec -ti vault-0 -n $KUBENAMESPACE -- vault secrets list

In [ ]:
# Sidecar read from pod file
kubectl exec -ti vault-0 -n $KUBENAMESPACE -c debug-container -- cat /vault/audit/vault.log

## Clean Up

In [ ]:
# Clean up temp files
rm init.json
rm token.json

# Disable file audit device
vault audit disable file

# Remove the NodePort
kubectl delete svc port-vault-svc -n $KUBENAMESPACE 

# Delete Vault cluster
helm delete vault -n $KUBENAMESPACE

# Delete the Vault for Transit Auto Unseal
helm delete vault-transit -n $KUBENAMESPACE

# Clear Vault PVCs
kubectl -n $KUBENAMESPACE delete pvc --all 

# Delete kind cluster
kind delete cluster --name vault

# Appendix - Other Useful Commands

In [ ]:
# Optional: Turn on the file audit device, this allows you to keep a detailed log of all requests to Vault
vault audit enable file file_path=/vault/audit/vault_audit.log

In [ ]:
# Optional: view pod logs
kubectl logs vault-0 -n $KUBENAMESPACE

In [ ]:
# Optional: view pod details
kubectl describe pod vault-0 -n $KUBENAMESPACE

In [ ]:
# Optional: Add metrics-server to be able to view CPU and memory usage
helm repo add metrics-server https://kubernetes-sigs.github.io/metrics-server/
helm repo update
helm upgrade --install --set args={--kubelet-insecure-tls} metrics-server metrics-server/metrics-server --namespace kube-system

In [ ]:
# Optional: You can use k9s to view your pods.
# You can also use the following commands to see the utlization on your nodes/pods
kubectl top nodes
echo
kubectl top pod -n $KUBENAMESPACE

In [ ]:
# To get a shell into a Vault pod
echo $KUBENAMESPACE
kubectl exec -ti vault-0 -n $KUBENAMESPACE -- /bin/sh
#kubectl exec -ti vault-0 -n vault-ns -- /bin/sh

In [ ]:
# Show ConfigMap resources for Vault
kubectl get configmap -n $KUBENAMESPACE -o=yaml

In [ ]:
# Show vault-config details
kubectl describe configmaps vault-config -n $KUBENAMESPACE

In [ ]:
# Show Vault pod details
kubectl describe pod vault-0 -n $KUBENAMESPACE

In [ ]:
# Show Persistent Volume Claims in use by Vault
kubectl get pvc -n $KUBENAMESPACE -o=yaml